In [49]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import numpy as np
import matplotlib.pyplot as plt
from fuzzy_layer import FuzzyLayer
import matplotlib.pyplot as plt


np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

timeseries_length = 80

Сгенерируем набор синусоид с двумя различными параметрами - фазой и частотой, и разметим классы:

0. Синус с периодом 2 pi
1. Косинус с периодом 2 pi
2. Синус с периодом 4 pi
3. Косинус с периодом 4 pi
4. Нет разметки

In [50]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, samples_count, timeseries_length):
        'Initialization'
        self.samples = samples_count
        self.timeseries_length = timeseries_length
    
    def __len__(self):
        'Denotes the total number of samples'
        return self.samples
    
    def __getitem__(self, index):
        'Generates one sample of data'
        
        phase_state = np.random.randint(0,11)
        freq_state = np.random.randint(0,11)

        freq = freq_state / 10.0
        phase = phase_state / 10.0
        t = np.linspace(0.0, 6.0 * np.pi, self.timeseries_length)
        X =(1.0 + np.sin(phase * np.pi/2.0 - freq * t)) / 2.0

        X = X.type('torch.FloatTensor')
        
        y = 4
        if phase_state == 0 and freq_state == 0:
            y = 0
        elif phase_state == 10 and freq_state == 0:
            y = 1
        elif phase_state == 0 and freq_state == 10:
            y = 2
        elif phase_state == 10 and freq_state == 10:
            y = 3

        return X, y
    
training_set = Dataset(1000, timeseries_length)
train_gen = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle = True, num_workers = 0, drop_last=True)

testing_set = Dataset(128, timeseries_length)
test_gen = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle = True, num_workers = 0, drop_last=True)

Определим структуру VAE для одномерного сигнала

In [57]:
class VAE(nn.Module):

    def __init__(self):
        super(VAE, self).__init__()
        
        self.latent_dimension = 2

        self.convolutions = nn.Sequential(
            nn.Conv1d(1, 16, 5),
            nn.BatchNorm1d(16),
            nn.Conv1d(16, 32, 5),
            nn.BatchNorm1d(32),
            nn.Flatten()
        )

        self.z_mean = nn.Sequential(
            nn.Linear(2304, self.latent_dimension),
            nn.BatchNorm1d(self.latent_dimension),
            nn.ReLU()
        )

        self.z_logvar = nn.Sequential(
            nn.Linear(2304, self.latent_dimension),
            nn.BatchNorm1d(self.latent_dimension),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.latent_dimension, 2304),
            nn.BatchNorm1d(2304),
            nn.Unflatten(1, [32, 72]),
            nn.ConvTranspose1d(32,16,5),
            nn.BatchNorm1d(16),
            nn.ConvTranspose1d(16,1,5),
            nn.Flatten()
        )

    def reparam_trick(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def encode(self, x):
        conv = self.convolutions.forward(x)
        z_mean = self.z_mean.forward(conv)
        z_logvar = self.z_logvar.forward(conv)

        z = self.reparam_trick(z_mean, z_logvar)
        return (z_mean, z_logvar, z)
        
    
    def forward(self, x):
        (_, _, z) = self.encode(x)

        return self.decoder(z)
        
model = VAE().to(device)
model

VAE(
  (convolutions): Sequential(
    (0): Conv1d(1, 16, kernel_size=(5,), stride=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Flatten(start_dim=1, end_dim=-1)
  )
  (z_mean): Sequential(
    (0): Linear(in_features=2304, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (z_logvar): Sequential(
    (0): Linear(in_features=2304, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=2304, bias=True)
    (1): BatchNorm1d(2304, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Unflatten(dim=1, unflattened_size=[32, 72])
    (3): ConvT

In [67]:
input = torch.randn(10, 1, timeseries_length, device=device)

output = model.forward(input)

output.shape

torch.Size([10, 80])

In [56]:
2304/32

72.0